# Model to predict cavities at grid-points in a water box
## Approach:
We run a 50 nanosecond MD simulation of tip3p water in a 5nm by 5nm by 5nm box, with a time step of 2fs, with frames being saved after every 500 timesteps. This gives us 50000 frames in all.

We intend to supply the position coordinates of the water-oxygen atoms in each frame as inputs and for each frame a sequence of grid-points labeled as "cavity ~ 1" and "no-cavity ~ 0" as the output.
For each frame then, the network is then expected to do a binary classification for each of the output neurons (which is a sequence of the grid points) based on the input coordinates of all the water(oxygen) molecules in that frame.

In [1]:
import numpy as np
import tensorflow as tf
import keras
import os
import MDAnalysis as mda

Using TensorFlow backend.


## Data Processing

This involved two broad steps:
1. Input data preparation
2. Label Generation

### Inputs:
We dump all the individual saved frames as separate ```.gro``` files. 
We used shell scripts to process the ```.gro``` files into ```.dat``` files. The rows containing the water-hydrogen coordinates were removed (under the assumption that location of a water molecule is given by the location of the water molecules); and changed the extension to ```.dat```.
_Note: Make sure there are six distinct columns throughout the data file, otherwise import using ```np.loadtxt``` for last three columns will fail. Here, for atoms near the end of the file, the atom index becomes large and the space delimiter between the 3rd and 4th column disappears._

In [2]:
# path where .dat files are located
org_path = 'path/to/location/of/dat/files/and/trajectory'
dat_files = []
# load .dat files into array dat_files
for filename in os.listdir(org_path):
    if filename.endswith(".dat"):
        fulldat = os.path.join(org_path, filename)
        dat_files.append(fulldat)
# check how many files were loaded        
print(len(dat_files))

inputlist = []
# load columns of coordinates into inputlist
for item in dat_files:
    inputlist.append((np.loadtxt(item, usecols = (3,4,5,))))
print("Done.")

11988
Done.


In [3]:
# checks to see if the load worked correctly
print(len(inputlist))
print(np.ndim(inputlist))
print(inputlist[0])

11988
3
[[0.629 3.899 3.261]
 [2.18  3.632 2.952]
 [3.526 4.416 0.836]
 ...
 [3.447 4.65  0.503]
 [4.283 4.289 0.246]
 [4.318 3.076 1.07 ]]


### Labels:

Python Package ```MDAnalysis``` was used to manipulate trajectory data and obtain the presence of cavities at specific grid points.


In [4]:
# create mda universe object with the topology and trajectory
u = mda.Universe('run.tpr','run.xtc')

In [20]:
# create labels for grid points
# label contains cavity labels over all the frames
label =[]
# loop over frames in trajectory
for ts in u.trajectory:
    # array of cavity labels for each frame
    cavity =[]
    # grid size = 5 specified in the i,j,k loops
    for i in range(5) :
        for j in range(5) :
            for k in range(5) :
                # manipulation to prepare selection
                si = str(i) + ' '
                sj = str(j) + ' '
                sk = str(k) + ' '
                gridpoint = si + sj + sk
                selection = 'point ' + gridpoint + '2.0'
                # check if any items in selection and assign label accordingly
                if (len(u.select_atoms(selection))== 0):
                    cavity.append(1)
                else:
                    cavity.append(0)
    # for each frame append cavity array into label array
    
    label.append(cavity)
print("Done.")

Done.


In [21]:
# check to see labels worked correctly
np.shape(label)

(11988, 125)

Input is now loaded into ```inputlist``` and labels into ```label```. _Note: Vectorization and reshaping are still needed at this point befor putting into the model._

In [22]:
data = np.asarray(inputlist).astype('float32')

In [23]:
labels = np.asarray(label).astype('float32')

In [24]:
data.shape

(11988, 4055, 3)

In [25]:
train_data = np.reshape(data[:10000],(-1,4055*3))
train_data = train_data / 5
train_data.shape

(10000, 12165)

In [26]:
train_labels = labels[:10000]
train_labels.shape

(10000, 125)

In [27]:
test_data = np.reshape(data[10000:],(-1,4055*3))
test_data = test_data / 5
test_data.shape

(1988, 12165)

In [28]:
test_labels = labels[10000:]
test_labels.shape

(1988, 125)

In [41]:
from keras import models
from keras import layers

In [54]:
network = models.Sequential()
network.add(layers.Dense(256,activation='relu',input_shape=(12165,)))
network.add(layers.Dense(256,activation='relu'))
network.add(layers.Dense(256,activation='relu'))
network.add(layers.Dense(125,activation='sigmoid'))

In [55]:
network.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])

In [56]:
# history = network.fit(train_data, train_labels, epochs= ,batch_size=, validation_data=())
network.fit(train_data, train_labels, epochs=10 , shuffle=True, batch_size=128)

Epoch 1/10
10000/10000 [==============================] - 1s 64us/step - loss: 0.2507 - acc: 0.9402
Epoch 2/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1959 - acc: 0.9547
Epoch 3/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1915 - acc: 0.9547
Epoch 4/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1900 - acc: 0.9547
Epoch 5/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1885 - acc: 0.9547
Epoch 6/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1876 - acc: 0.9547
Epoch 7/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1874 - acc: 0.9547
Epoch 8/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1871 - acc: 0.9547
Epoch 9/10
10000/10000 [==============================] - 0s 31us/step - loss: 0.1869 - acc: 0.9547
Epoch 10/10
10000/10000 [==============================] - 0s 32us/step - loss: 0.1867 - acc: 0.9547

In [57]:
test_loss, test_acc = network.evaluate(test_data, test_labels)
print('test_acc:', test_acc)

1988/1988 [==============================] - 0s 90us/step
test_acc: 0.9548450716784302
